In [1]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

In [2]:
import math

def return_dict_summary_fixed(benchmark_name="cg", fixed="class"):
    if (fixed == "class"):
        fixed_ = return_fixed_class(BenchMark=benchmark_name)
    else:
        fixed_ = return_fixed_process(BenchMark=benchmark_name)
    summary_fixed_ = pd.concat(fixed_ , axis=1)
    columns = summary_fixed_.columns.to_numpy()
    index = summary_fixed_.index.to_numpy()
    if(fixed == "class"):
        dict_summary_fixed_ = {"processes":columns}
    else:
        dict_summary_fixed_ = {"class":columns}
    for index_name in index:
        dict_summary_fixed_[index_name] = summary_fixed_.T[index_name].to_numpy()
    return dict_summary_fixed_

def return_non_NaN_list(target_list):
    for i in range(len(target_list)):
        if (math.isnan(target_list[i])):
            target_list[i] = 0
    return target_list

def does_include_nan(target_list):
    for i in range(len(target_list)):
        if(math.isnan(target_list[i])):
            return True
    return False

In [3]:
def generateScoreTable(benchmark_name="cg"):
    list_ScoreTable = []
    dict_summary_fixed_class = return_dict_summary_fixed(benchmark_name=benchmark_name, fixed="class")
    raw_x = dict_summary_fixed_class["processes"]
#     print(f"raw_x : {raw_x}, benchmark : {benchmark_name}")
    for content in dict_summary_fixed_class:
        if(content == "processes"):
            continue
        raw_y = dict_summary_fixed_class[content]
        if(does_include_nan(raw_y)):
            continue
#         print(f"raw_y(={content}) : {raw_y}")
        # 線形モデル
        model_lin = ModelLin(raw_x, raw_y, benchmark_name, content)
        model_lin.calc_lr()
        model_lin.calc_r2_score()
#         model_lin.plot_graph()
        model_lin.calc_mae_score()
        model_lin.calc_mse_score()
        model_lin.calc_rmse_score()
        model_lin.calc_mape_score()
        # logモデル
        model_log10 = ModelLog10(raw_x, raw_y, benchmark_name, content)
        model_log10.calc_lr()
        model_log10.calc_r2_score()
#         model_log10.plot_graph()
        model_log10.calc_mae_score()
        model_log10.calc_mse_score()
        model_log10.calc_rmse_score()
        model_log10.calc_mape_score()
        # 反比例モデル
        model_ip = ModelIP(raw_x, raw_y, benchmark_name, content)
        model_ip.calc_lr()
        model_ip.calc_r2_score()
#         model_ip.plot_graph()
        model_ip.calc_mae_score()
        model_ip.calc_mse_score()
        model_ip.calc_rmse_score()
        model_ip.calc_mape_score()
        
        list_ScoreTable.append([content, model_lin.mape_score, model_log10.mape_score, model_ip.mape_score])
    df_ScoreTable = pd.DataFrame(list_ScoreTable)
    df_ScoreTable.columns = ["", "x mape", "logx mape", "1/x mape"]
    df_ScoreTable.set_index("",inplace=True)
    df_ScoreTable.to_csv("./tmp_GenerateScoreTable/"+benchmark_name+".csv")

generateScoreTable("bt")

In [4]:
for benchmark in benchmarks:
    # 以下の「benchmark == "XXX"」はNaNがデータとして存在するものを飛ばすために存在
    if(benchmark == "bt"):
        continue
    elif(benchmark == "ft"):
        continue
#     elif(benchmark == "is"):
#         continue
#     elif(benchmark == "mg"):
#         continue
#     elif(benchmark == "sp"):
#         continue
#     elif(benchmark == "ft"):
#         continue
    generateScoreTable(benchmark)